Now we model subgroups of families, workplaces etc., which will determine the probability of exposure. 
We will base this on the earlier described SIR model. 

In [1]:
# Initialization
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
# Define number of people (you may change here)
n = 1000

# Define number of families (you may change here)
n_families = 10
family_types = np.arange(1,n_families+1)

# Define work types (you may change here)
work_types = np.array(["BYG", "MMC", "KBS", "WIND"])
work_probs = np.array([0.1, 0.2, 0.4, 0.3])
n_work_types = len(work_types)

# Define vector of states
states = np.repeat("S",n)
states[0] = "I"
states_matrix = np.copy([states])


states_matrix = np.vstack((states_matrix, states ))
print(states_matrix)

# Probabilities of entering 
prob_S_I = 0.9
prob_I_R = 0.3

n_S = np.array([len(np.where(states == "S")[0])])
n_I = np.array([len(np.where(states == "I")[0])])
n_R = np.array([len(np.where(states == "R")[0])])


# Generate workplaces
workplaces_sorted = np.sort(np.random.choice(work_types,n, p = work_probs))
counts = np.unique(workplaces_sorted, return_counts = True)[1]
workplaces_counts_sorted = np.repeat(counts, counts)
random_positions = np.random.choice(np.arange(0,n), size = n, replace=False)

# Update workplaces based on positions
workplaces = workplaces_sorted[random_positions]
workplaces_counts = workplaces_counts_sorted[random_positions]

# Update families based on positions
families = np.sort(np.random.choice(family_types, n))
counts = np.unique(workplaces_sorted, return_counts = True)[1]
families_counts = np.repeat(counts, counts)

while ("I" in states):

    # Get positions of S, I, R
    S_index = np.where(states == "S")[0]
    I_index = np.where(states == "I")[0]
    R_index = np.where(states == "R")[0]

    # Count infected at each worktype
    work_I = np.zeros(n)
    for work in work_types:
        work_index = np.where(workplaces == work)[0]
        work_I[work_index] += len(np.where((workplaces == work) & (states == "I"))[0])
        
    # Count infected at each family  
    family_I = np.zeros(n)
    for family in family_types:
        family_index = np.where(families == family)[0]
        family_I[family_index] += len(np.where((families == family) & (states == "I"))[0])
        
    # Calculate probability
    prob_vector = prob_S_I * ((work_I + family_I) / (workplaces_counts + families_counts))

    
    for pos in S_index:
        states[pos] = np.random.choice(np.array(["S","I"]),p = np.array([1-prob_vector[pos], prob_vector[pos]]))
        
        

    # Update infected
    states[I_index] = np.random.choice(np.array(["I", "R"]),p = np.array([1-prob_I_R, prob_I_R]), size = len(I_index))
    
    
    # Update states_matrix
    states_matrix = np.vstack((states_matrix, states ))
    
    n_S = np.concatenate([n_S, np.array([len(np.where(states == "S")[0])])])
    n_I = np.concatenate([n_I, np.array([len(np.where(states == "I")[0])])])
    n_R = np.concatenate([n_R, np.array([len(np.where(states == "R")[0])])])

[['I' 'S' 'S' ... 'S' 'S' 'S']
 ['I' 'S' 'S' ... 'S' 'S' 'S']]


In [ ]:
plt.figure()

for fam_number in family_types[0:6]:
    n_fam = np.array([])
    for row in states_matrix:
        f = len(np.where((row == "I")& (families == fam_number))[0])
        n_fam = np.concatenate([n_fam, np.array([f])])
        
    n_frac_fam = n_fam/np.sum(families == fam_number)
    
    plt.plot(n_frac_fam, label = str(fam_number))

plt.legend()
plt.ylabel("Fraction of infected people in family")
plt.xlabel("Days")
plt.show()


plt.figure()

for work_string in work_types:
    n_work = np.array([])
    for row in states_matrix:
        w = len(np.where((row == "I")& (workplaces == work_string))[0])
        n_work = np.concatenate([n_work, np.array([w])])
    
    n_frac_work = n_work/np.sum(workplaces == work_string)
        
    plt.plot(n_frac_work, label = work_string)

plt.legend()
plt.ylabel("Fraction of infected people in work group")
plt.xlabel("Days")
plt.show()